In [3]:
from rfid_app import db
from rfid_app.models import Student, Bus, Route, TripHistory, BusStop, bus_and_routes, bus_stops_and_routes
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker
import pandas as pd

In [4]:
engine = create_engine('postgresql://localhost:5432/rfid_app')
Session = sessionmaker(bind=engine)
session = Session()

In [5]:
conn_str='postgresql://localhost:5432/rfid_app'

In [6]:
pd.read_sql('SELECT students.first_name, students.last_name, students.present, \
                bus_stops.name AS bus_stop, routes.name AS route_name, buses.number AS bus_number\
            FROM bus_stops, bus_stop_route, students, routes, buses, bus_route \
            WHERE students.home_stop_id=bus_stop_route.stop_id \
            AND students.home_stop_id=bus_stops.id \
            AND bus_stop_route.route_id=routes.id \
            AND bus_route.bus_id=buses.id \
            AND bus_stop_route.route_id=bus_route.route_id'
            , con=conn_str)

,first_name,last_name,present,bus_stop,route_name,bus_number
0,bob,boblast,False,mercer,blue,45
1,sara,omaley,False,greenlake,green,321
2,steve,mcqueen,False,westlake,green,321
3,sue,test,False,queenanne,blue,45
4,tom,cat,False,caphill,orange,99


In [7]:
pd.read_sql('SELECT * \
            FROM bus_stops, bus_stop_route, bus_route, buses \
            WHERE bus_stops.id=bus_stop_route.stop_id \
            AND bus_route.route_id=bus_stop_route.stop_id \
            AND buses.id=bus_route.bus_id \
            AND buses.number=45'
            , con=conn_str)

,id,name,stop_id,route_id,route_id,bus_id,id,number
0,1,mercer,1,1,1,2,2,45


In [29]:
bus_stop = pd.read_sql('SELECT bus_stops.name AS bus_stop \
            FROM bus_stops, buses, bus_stop_route, bus_route \
            WHERE buses.number={} \
            AND bus_stops.id=bus_stop_route.stop_id \
            AND bus_stop_route.route_id=bus_route.route_id \
            AND bus_route.bus_id=buses.id'.format(45)
            , con=conn_str)

In [36]:
b =bus_stop['bus_stop']

In [37]:
b.tolist()

['mercer', 'queenanne']

In [42]:
Bus.query.filter_by(number=45).first().number

45